**TIME SERIES MODEL - ARIMA**

In [ ]:
## Importing packages
library(tidyverse) # metapackage with lots of helpful functions
library(lubridate)
library(forecast)
library(tseries)
library(fBasics)
library(zoo)
library(fUnitRoots)
library(corrplot)
library(TSA)
library(pastecs)

## Reading in files
# You can access files from datasets you've added to this kernel in the "../input/" directory.
# You can see the files added to this kernel by running the code below. 

list.files(path = "../input/")
d_ns<-read.csv("../input/df_solararray_complete.csv", header=TRUE)
head(d_ns)

In [ ]:
# Let's replace precipitation and pressure with global mean since lots of these 
#pc_mdn <- mean(d_ns$Precipitation, na.rm=TRUE)
#d_ns$Precipitation[is.na(d_ns$Precipitation)] = pc_mdn

#ps_mdn <- mean(d_ns$Pressure, na.rm=TRUE)
#d_ns$Pressure[is.na(d_ns$Pressure)] = ps_mdn

summary(d_ns)

In [ ]:
####### Time Series Models without transformation or normalization ########
# let's split the data into train and validation for this problem
# Training = 80% and Validation = 20%

smp_size<-floor(0.8*nrow(d_ns))
train<-d_ns[1:smp_size,]
validation<-d_ns[(smp_size+1):nrow(d_ns),]

Location=train[,2]
Year=train[,3]
Month=train[,4]
Day=train[,5]
Hour=train[,6]
Cloud_Cover_Fraction=train[,7]
Dew_Point=train[,8]
Humidity_Fraction=train[,9]
Precipitation=train[,10]
Pressure=train[,11]
Temperature=train[,12]
Visibility=train[,13]
Wind_Speed=train[,14]
Solar_Elevation=train[,15]
Electricity_KW_HR=train[,16]



In [ ]:
# based on the dickey fuller test, we canot reject non-stationarity since p value is less than 0.05
adfTest(Electricity_KW_HR, lags=5, type=c('c'))

In [ ]:
dx=diff(Electricity_KW_HR, 24)
acf(dx, lag.max=100)
ddx=diff(diff(Electricity_KW_HR, 24))
acf(ddx, lag.max=100)
dddx=diff(diff(diff(Electricity_KW_HR, 24)))
acf(dddx, lag.max=100)  # so based on third difference we can see that the dacay happend quickly at 3rd difference and became stationary

In [ ]:
pacf(dddx)
eacf(dddx)
# based on the eacf plot, we can see the ar will be 0, and ma will be 2 with difference of 3. So let's try our first model 

In [ ]:
# based on Dickey-Fuller Test, we can reject null hypothesis of stationarity since p-value is less than 0.05
adfTest(Electricity_KW_HR, lags=3, type=c('c'))  

In [ ]:
acf(diff(Electricity_KW_HR))
#acf(log(diff(lne)), lag.max=100)

In [ ]:
ndiffs(Electricity_KW_HR)
plot(ddx)

In [ ]:
d1

In [ ]:
# Let's do it manually
#m<-arima(Electricity_KW_HR, order=c(0,1, 2), xreg=d1)
#adfTest(m$residuals)

In [ ]:
#d_ns$date<-ymd_hms(paste(d_ns$Year, d_ns$Month, d_ns$Day, d_ns$Hour-1, '00', '00', sep=':'))

#mydata$datetime<-ymd_hms(paste(mydata$year, mydata$month, mydata$day, mydata$hour, mydata$minute, mydata$second, sep="-"))
#str(mydata)
#head(d_ns,25)   

In [ ]:
# let's plot a decomposition plot

#df_ts<-ts(d_ns$Hour, d_ns$Electricity_KW_HR, frequency=12)
#plot(decompose(df_ts))

# Try frequency of 24
df.ts1 = ts(Electricity_KW_HR, start=1, frequency=24) 
plot(df.ts1, col='orange')

# since there are yearly data here taken every hour, we have total
# frequency of 8766
df.ts2 = ts(Electricity_KW_HR, start=1, frequency=8766) 
plot(df.ts2, col='blue')

plot(decompose(df.ts1),col='orange')
plot(decompose(df.ts2),col='blue')

I looks like trend is flat.  There appears to be seasonality in this series since the y-axis of the seanal graph shows range from -25000 to 65000 which is similar to the range for the Electricity_KW_HR y-axis.

In [ ]:
#acf(Electricity_KW_HR, lag.max=200, main='ACF plot of the Electricity (KWh) Time Series')
#pacf(Electricity_KW_HR, lag.max=200, main="PACF Plot of Electricity (KWh)")
#eacf(Electricity_KW_HR)

Looks like the ACF plot shows slow decaying series so the series is non-stationary.
Based on the ACF and PACF function, it looks like it's not AR or MA model. It's a blend of both AR and MA model. Therefore, it's an ARMA model. There are more than 22 p values in AR model based on the PACF model. There are more than 14 values for q in MA model based on the ACF plot.  
Since both ACF and PACF plots shows us the series moving, we will take the first difference of the series and see if it is stationary.

Since it's hard to tell if there is any difference we will perform differencing and check the stationarity

In both ACF and PACF plots of the first difference we can tee this there is a seasonal component repeated every 24 lags

Let's compute linear regression model of Electricity_KW_HR vs all other predictors except for the dates

In [ ]:
# model 0
head(d_ns[7:16])
m0<-lm(Electricity_KW_HR~Cloud_Cover_Fraction+Dew_Point+Humidity_Fraction+Precipitation+Pressure+Temperature+Visibility+Wind_Speed+Solar_Elevation, data=d_ns)
summary(m0)

# based on the results, only the intercept, cloud_Cover_Fraction, Dew_Point, Pressure, Temperature, Visibility, Solar_Elevations are significatnt
# we will build second model with those variables

In [ ]:
# based on Dickey-Fuller Test, we can reject null hypothesis of stationarity since p-value is less than 0.05
library(fUnitRoots)
adfTest(m0$residuals)  

In [ ]:
# model 1
# let's remove the highly unsignificant variable, Humidity_Fraction to see if our model changes
m1<-lm(Electricity_KW_HR~Cloud_Cover_Fraction+Dew_Point+Precipitation+Pressure+Temperature+Visibility+Wind_Speed+Solar_Elevation, data=d_ns)
summary(m1)
adfTest(m1$residuals) # based on Dickey-Fuller Test, we can reject null hypothesis of stationarity since p-value is less than 0.05

In [ ]:
# model 2
# let's remove the next highly unsignificant variable, Precipitation to see if our model changes
m2<-lm(Electricity_KW_HR~Cloud_Cover_Fraction+Dew_Point+Pressure+Temperature+Visibility+Wind_Speed+Solar_Elevation, data=d_ns)
summary(m2)
adfTest(m2$residuals) # based on Dickey-Fuller Test, we can reject null hypothesis of stationarity since p-value is less than 0.05

In [ ]:
# model 3
# let's remove the next highly unsignificant variable, Wind_Speed to see if our model changes
m3<-lm(Electricity_KW_HR~Cloud_Cover_Fraction+Dew_Point+Pressure+Temperature+Solar_Elevation, data=d_ns)
summary(m3)
adfTest(m3$residuals) # based on Dickey-Fuller Test, we can reject null hypothesis of stationarity since p-value is less than 0.05

In [ ]:
# Diagnostic plots
plot(m3$residuals, type='l')
acf(coredata(m3$residuals, max.lags=50))

Based on the residual plot, it looks like the residuals are stationary around zero. However, it also looks like lots of volatility in the residual plot  
However, the autocorrelation (acf) plot of the residuls shows slow decalying plot. Also, it shows seasonality since the plot is decaying with sinusoidal wave.

In [ ]:
# model 4
# on first differenced data create 
Cloud_Cover_Fraction_rt=diff(Cloud_Cover_Fraction)/Cloud_Cover_Fraction[-1]
Dew_Point_rt=diff(Dew_Point)/Dew_Point[-1]
#Humidity_Fraction_rt=diff(Humidity_Fraction)/Humidity_Fraction[-1]
#Precipitation_rt=diff(Precipitation)/Precipitation[-1]
Temperature_rt=diff(Temperature)/Temperature[-1]
#Visibility_rt=diff(Visibility)/Visibility[-1]
Wind_Speed_rt=diff(Wind_Speed)/Wind_Speed[-1]
Pressure_rt=diff(Pressure)/Pressure[-1]
Solar_Elevation_rt=diff(Solar_Elevation)/Solar_Elevation[-1]
Electricity_KW_HR_rt=diff(Electricity_KW_HR)/Electricity_KW_HR[-1]

In [ ]:
sum(is.na(Cloud_Cover_Fraction_rt))

In [ ]:
par(mfrow=c(2,3))
plot(Cloud_Cover_Fraction_rt, type='l', main="Cloud Cover Fraction Return")
plot(Dew_Point_rt, type='l', main="Dew Point Return")
#plot(Humidity_Fraction_rt, type='l', main="Humidity Fraction Return")
#plot(Precipitation_rt, type='l', main="Precipitation Return")
plot(Wind_Speed_rt, type='l', main="Wind Speed Return")
plot(Solar_Elevation_rt, type='l', main="Solar Elevation Return")
par(mfrow=c(2,2))
plot(Pressure_rt, type='l', main="Pressure Return")
#plot(Visibility_rt, type='l', main="Visibility Return")
plot(Temperature_rt, type='l', main="Temperature Return")
plot(Electricity_KW_HR_rt, type='l', main="Electricity (KWh) Return")

In [ ]:
# regression model of consumprt vs pers_incrt and unemprt with ARMA errors
# STEP 1: fit simple regression model with no intercept
# m4 = lm(Electricity_KW_HR_rt~Cloud_Cover_Fraction_rt+Dew_Point_rt+Pressure_rt+Temperature_rt+
#        Wind_Speed_rt+Solar_Elevation_rt)
# summary(m4)
# print("The standard deviation of the residual is: ")
# stdev(m1$residuals)   # Recall that the standard error is the stdev of the residuals
# plot(m4$residuals, type='l')  



In [ ]:
print('Box Test of m1$residuals:')
Box.test(m1$residuals, type="Ljung-Box")  # Test is marginal at the .1 confidence leve
                                          # So, for the sake of argument, let's fit an arma model

print('Print acf and pacf of m1$residuals')
Acf(m1$resid)    # still there is slow decay identical to the original resid from model 0.
Pacf(m1$resid)   # same graph as the residual for model 0

In [ ]:
# let's do auto arima to see what model does it suggest on original data based on BIC criteria with all independent variables
d1<-cbind(Cloud_Cover_Fraction, Dew_Point,Humidity_Fraction,Precipitation, Pressure, Temperature, Visibility, Wind_Speed, Solar_Elevation)

m4=auto.arima(Electricity_KW_HR, xreg = d1, ic='bic')
m4

acf(m4$residuals, na.action = na.pass, lag.max=100)
library(lmtest)
coeftest(m4)
Box.test(m4$residuals,type="Ljung-Box")
# Based on model x2 and x3 values, we can see x2 aic is -1125 and x3 aic is -1128, but the x2 only has 
# two significant terms (ma1 and intercept) while x3 has three terms ar1, ma1, and intercept. 
# Therefore, we can go with parsemonious model x2.  This model is adequate since the ACF and PACF 
# shows no significant lag and the box.test shows p-value of 0.98 which means we can accept the 
# null hypothesis that what's there is a white noise  

In [ ]:
# let's do auto arima to see what model does it suggest based on few independent variables from model m3
# We will employ BIC criteria with all independent variables
d2<-cbind(Cloud_Cover_Fraction, Dew_Point, Pressure, Temperature, Solar_Elevation)

m5=auto.arima(Electricity_KW_HR, xreg = d2, ic='bic')
m5

acf(m5$residuals, na.action = na.pass, lag.max=100)
library(lmtest)
coeftest(m5)
Box.test(m5$residuals,type="Ljung-Box")
# Based on model x2 and x3 values, we can see x2 aic is -1125 and x3 aic is -1128, but the x2 only has 
# two significant terms (ma1 and intercept) while x3 has three terms ar1, ma1, and intercept. 
# Therefore, we can go with parsemonious model x2.  This model is adequate since the ACF and PACF 
# shows no significant lag and the box.test shows p-value of 0.98 which means we can accept the 
# null hypothesis that what's there is a white noise  

In [ ]:
# now let's employ AIC criteria with minimum number of independent variables from m3
d3<-cbind(Cloud_Cover_Fraction, Dew_Point, Pressure, Temperature, Solar_Elevation)

m6=auto.arima(Electricity_KW_HR, xreg = d3, ic='aic')
m6

acf(m6$residuals, na.action = na.pass, lag.max=100)
library(lmtest)
coeftest(m6)
Box.test(m6$residuals,type="Ljung-Box")
# Based on model x2 and x3 values, we can see x2 aic is -1125 and x3 aic is -1128, but the x2 only has 
# two significant terms (ma1 and intercept) while x3 has three terms ar1, ma1, and intercept. 
# Therefore, we can go with parsemonious model x2.  This model is adequate since the ACF and PACF 
# shows no significant lag and the box.test shows p-value of 0.98 which means we can accept the 
# null hypothesis that what's there is a white noise  

In [ ]:
# let's now forcast and get the accuracy on the training

Location=validation[,2]
Year=validation[,3]
Month=validation[,4]
Day=validation[,5]
Hour=validation[,6]
Cloud_Cover_Fraction=validation[,7]
Dew_Point=validation[,8]
Humidity_Fraction=validation[,9]
Precipitation=validation[,10]
Pressure=validation[,11]
Temperature=validation[,12]
Visibility=validation[,13]
Wind_Speed=validation[,14]
Solar_Elevation=validation[,15]
Electricity_KW_HR=validation[,16]
r1<-cbind(Cloud_Cover_Fraction, Dew_Point,Humidity_Fraction,Precipitation, Pressure, Temperature, Visibility, Wind_Speed, Solar_Elevation)
r2<-cbind(Cloud_Cover_Fraction, Dew_Point, Pressure, Temperature, Solar_Elevation)
r3<-cbind(Cloud_Cover_Fraction, Dew_Point, Pressure, Temperature, Solar_Elevation)

In [ ]:
f4<-forecast(m4, xreg=r1)
accuracy(f4, Electricity_KW_HR)

f5<-forecast(m5, xreg=r2)
accuracy(f5, Electricity_KW_HR)

f6<-forecast(m6, xreg=r3)
accuracy(f6, Electricity_KW_HR)